<a href="https://colab.research.google.com/github/thinothw/DFDS-Final-Project/blob/main/Phase01_Sandbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install retina-face opencv-python

In [1]:
# Clean, compatible install block
!pip uninstall -y numpy -q
!pip install -q numpy==1.26.4
!pip install -q datasets==2.18.0
!pip install -q facenet-pytorch==2.5.3
!pip install -q opencv-python==4.8.0.76

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 62.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tobler 0.13.0 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
pytensor 2.38.0 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.13.0.92 requires numpy>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
xarray-einstats 0.10.0 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.13.0.92 requires numpy>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
j

In [2]:
# INFRASTRUCTURE - Connect to Drive

from google.colab import drive
import os

print("Requesting Google Drive access...")
drive.mount('/content/drive')

# Double check the project folder exists
base_path = '/content/drive/MyDrive/Deepfake_Honours_Project/Sandbox_10K'
if os.path.exists(base_path):
    print(f"✅ Connection Stable! Project folder found at: {base_path}")
else:
    print(f"⚠️ Warning: Base path not found. Check your Drive folder name!")

Requesting Google Drive access...
Mounted at /content/drive
✅ Connection Stable! Project folder found at: /content/drive/MyDrive/Deepfake_Honours_Project/Sandbox_10K


In [3]:
import numpy
import pandas
import torch
print("✅ All core imports successful. The environment is stable.")

✅ All core imports successful. The environment is stable.


In [4]:
# OpenFake Sandbox Dataset Downloader - 12,000 Images.
# Code ran with 0 saved errors!.

# 1. Core Imports
import os
import torch
import shutil
from datasets import load_dataset

# 2. GPU Verification
print("\nChecking GPU availability...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device detected: {device}")

# 3. Create High-Speed Local Folders
print("\nSetting up high-speed local storage...")
local_base = '/content/temp_raw'
local_real = os.path.join(local_base, 'real')
local_fake = os.path.join(local_base, 'fake')

os.makedirs(local_real, exist_ok=True)
os.makedirs(local_fake, exist_ok=True)

# 4. Connect to OpenFake Dataset
print("\nConnecting to OpenFake dataset stream...")
dataset_name = "ComplexDataLab/OpenFake"

try:
    openfake = load_dataset(dataset_name, split='train', streaming=True)
    print("Dataset connection successful.")
except Exception as e:
    raise RuntimeError(f"Dataset loading failed: {e}")

# 5. Download Buffer (6,000 Real + 6,000 Fake)
TARGET_BUFFER = 6000
real_count = 0
fake_count = 0
save_errors = 0
max_iterations = 80000
iteration_counter = 0

print(f"\nStarting MASS DOWNLOAD to local storage (Target: {TARGET_BUFFER} per class)...")

for item in openfake:
    iteration_counter += 1

    if iteration_counter > max_iterations:
        print("\n⚠️ Iteration cap exceeded. Stopping download.")
        break

    try:
        label = item['label']
        image = item.get('image', None) # Safely attempt to get the image

        # Safety 1: The Null Check
        if image is None:
            continue

        # Safety 2: Handle Integer Labels
        if isinstance(label, int):
            label = 'real' if label == 0 else 'fake'

        if label not in ['real', 'fake']:
            continue

        # Safety 3: Force RGB format
        if image.mode != 'RGB':
            image = image.convert('RGB')

        if label == 'real' and real_count < TARGET_BUFFER:
            image.save(os.path.join(local_real, f"raw_openfake_real_{real_count}.jpg"))
            real_count += 1
            if real_count % 1000 == 0:
                print(f"  -> Downloaded {real_count} / {TARGET_BUFFER} Real images...")

        elif label == 'fake' and fake_count < TARGET_BUFFER:
            image.save(os.path.join(local_fake, f"raw_openfake_fake_{fake_count}.jpg"))
            fake_count += 1
            if fake_count % 1000 == 0:
                print(f"  -> Downloaded {fake_count} / {TARGET_BUFFER} Fake images...")

        if real_count == TARGET_BUFFER and fake_count == TARGET_BUFFER:
            break

    except Exception as e:
        save_errors += 1
        print(f"Error at iteration {iteration_counter}: {e}")

print("\n=== Download Summary ===")
print(f"Real images saved locally: {real_count}")
print(f"Fake images saved locally: {fake_count}")
print(f"Save errors: {save_errors}")

# 6. Move to Google Drive
print("\nMoving files from local storage to Google Drive. Please wait...")
drive_base = '/content/drive/MyDrive/Deepfake_Honours_Project/Sandbox_10K/raw_data'
drive_real = os.path.join(drive_base, 'real')
drive_fake = os.path.join(drive_base, 'fake')

os.makedirs(drive_real, exist_ok=True)
os.makedirs(drive_fake, exist_ok=True)

# Copy everything over
shutil.copytree(local_real, drive_real, dirs_exist_ok=True)
shutil.copytree(local_fake, drive_fake, dirs_exist_ok=True)

# 7. Purge Local Temp Files
print("Cleaning up temporary local files to free space...")
shutil.rmtree(local_base)

print(" Mass download, Drive transfer, and cleanup completed successfully!")


Checking GPU availability...
Device detected: cpu

Setting up high-speed local storage...

Connecting to OpenFake dataset stream...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/206 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/206 [00:00<?, ?it/s]

Dataset connection successful.

Starting MASS DOWNLOAD to local storage (Target: 6000 per class)...
  -> Downloaded 1000 / 6000 Fake images...
  -> Downloaded 1000 / 6000 Real images...
  -> Downloaded 2000 / 6000 Real images...
  -> Downloaded 2000 / 6000 Fake images...


/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


  -> Downloaded 3000 / 6000 Real images...
  -> Downloaded 3000 / 6000 Fake images...
  -> Downloaded 4000 / 6000 Fake images...
  -> Downloaded 4000 / 6000 Real images...
  -> Downloaded 5000 / 6000 Fake images...
  -> Downloaded 5000 / 6000 Real images...
  -> Downloaded 6000 / 6000 Real images...
  -> Downloaded 6000 / 6000 Fake images...

=== Download Summary ===
Real images saved locally: 6000
Fake images saved locally: 6000
Save errors: 0

Moving files from local storage to Google Drive. Please wait...
Cleaning up temporary local files to free space...
 Mass download, Drive transfer, and cleanup completed successfully!


In [5]:
# PHASE 2 - RetinaFace V3

import os
import cv2
import numpy as np
from retinaface import RetinaFace
from PIL import Image
import warnings

warnings.filterwarnings("ignore")

print("Bouncer operating on: CPU (RetinaFace V4.3)\n")

# Paths
raw_real = os.path.join(base_path, 'raw_data/real')
raw_fake = os.path.join(base_path, 'raw_data/fake')

hq_real = os.path.join(base_path, 'processed_data_V4_3/real')
hq_fake = os.path.join(base_path, 'processed_data_V4_3/fake')

reject_real = os.path.join(base_path, 'processed_data_V4_3/rejected/real')
reject_fake = os.path.join(base_path, 'processed_data_V4_3/rejected/fake')

for path in [hq_real, hq_fake, reject_real, reject_fake]:
    os.makedirs(path, exist_ok=True)

# Blur Detection
def blur_score(pil_image):
    img = np.array(pil_image)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    return cv2.Laplacian(gray, cv2.CV_64F).var()

# RetinaFace Extraction Engine (Pragmatic Filters)
def hq_crop_engine_v4_3(
    img_path,
    save_path,
    reject_path,
    padding=25,
    min_face_size=30,      #  Lowered to allow gritty/low-res data
    min_face_ratio=0.005,  #  Lowered to 0.5% to allow wider shots
    blur_threshold=50,
    confidence_threshold=0.90
):
    try:
        faces = RetinaFace.detect_faces(img_path)

        if not isinstance(faces, dict) or len(faces) == 0:
            return False, "No Face Detected"

        largest_area = 0
        best_face = None

        for face in faces.values():
            box = face.get('facial_area', None)
            if box is None:
                continue

            area = (box[2] - box[0]) * (box[3] - box[1])

            if area > largest_area:
                largest_area = area
                best_face = face

        if best_face is None:
            return False, "No Valid Face Found"

        box = best_face['facial_area']
        confidence = best_face['score']

        if confidence < confidence_threshold:
            return False, f"Low Confidence ({confidence:.2f})"

        img_pil = Image.open(img_path).convert('RGB')
        width, height = img_pil.size

        face_w = box[2] - box[0]
        face_h = box[3] - box[1]
        face_area = face_w * face_h
        image_area = width * height

        if face_w < min_face_size or face_h < min_face_size:
            return False, f"Too Small Pixels ({int(face_w)}x{int(face_h)})"

        ratio = face_area / image_area
        if ratio < min_face_ratio:
            return False, f"Too Small Ratio ({ratio:.3f})"

        x_min = max(0, int(box[0]) - padding)
        y_min = max(0, int(box[1]) - padding)
        x_max = min(width, int(box[2]) + padding)
        y_max = min(height, int(box[3]) + padding)

        cropped_img = img_pil.crop((x_min, y_min, x_max, y_max))

        blur_val = blur_score(cropped_img)
        if blur_val < blur_threshold:
            return False, f"Blurred ({blur_val:.2f})"

        cropped_img.save(save_path)
        return True, "Success"

    except Exception as e:
        return False, str(e)

# Dataset Cleaner
def clean_dataset_v4_3(input_folder, output_folder, reject_folder, label):
    print(f"\n--- Cleaning {label} Folder ---")
    images = [f for f in os.listdir(input_folder) if f.endswith('.jpg')]

    success = 0
    rejected = 0
    reasons = {}

    for name in images:
        img_path = os.path.join(input_folder, name)
        save_path = os.path.join(output_folder, name)
        reject_path = os.path.join(reject_folder, name)

        ok, msg = hq_crop_engine_v4_3(img_path, save_path, reject_path)

        if ok:
            success += 1
            print(f"  [✓] EXTRACTED: {name}")
        else:
            rejected += 1
            print(f"  [X] REJECTED: {name} -> Reason: {msg}")
            reasons[msg] = reasons.get(msg, 0) + 1
            Image.open(img_path).save(reject_path)

    print(f"\n✅ FINAL STATS for {label}: {success} Valid | {rejected} Rejected")
    print("Rejection Breakdown:")
    for k, v in reasons.items():
        print(f"   {k} → {v}")
    print("="*40)

# Run
clean_dataset_v4_3(raw_real, hq_real, reject_real, "REAL")
clean_dataset_v4_3(raw_fake, hq_fake, reject_fake, "FAKE")

print("\n Phase 2.8 Complete! Check processed_data_V4_3")

Bouncer operating on: CPU (RetinaFace V4.3)


--- Cleaning REAL Folder ---
  [✓] EXTRACTED: test_openfake_real_0.jpg
  [✓] EXTRACTED: test_openfake_real_1.jpg
  [X] REJECTED: test_openfake_real_2.jpg -> Reason: Too Small Ratio (0.005)
  [✓] EXTRACTED: test_openfake_real_3.jpg
  [X] REJECTED: test_openfake_real_4.jpg -> Reason: No Face Detected
  [✓] EXTRACTED: test_openfake_real_5.jpg
  [✓] EXTRACTED: test_openfake_real_6.jpg
  [X] REJECTED: test_openfake_real_7.jpg -> Reason: No Face Detected
  [✓] EXTRACTED: test_openfake_real_8.jpg
  [X] REJECTED: test_openfake_real_9.jpg -> Reason: Too Small Ratio (0.002)
  [X] REJECTED: test_openfake_real_10.jpg -> Reason: Too Small Pixels (24x32)
  [X] REJECTED: test_openfake_real_11.jpg -> Reason: Too Small Pixels (24x24)
  [✓] EXTRACTED: test_openfake_real_12.jpg
  [✓] EXTRACTED: test_openfake_real_13.jpg
  [✓] EXTRACTED: test_openfake_real_14.jpg
  [✓] EXTRACTED: test_openfake_real_15.jpg
  [✓] EXTRACTED: test_openfake_real_16.jpg
  [✓] EXTRAC